In [1]:
# Deep Learning / Machine Learning
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from torchsummary import summary
import numpy as np

# EDA
import pandas as pd
import plotly.express as px


## Leitura dos dados

In [2]:
veiculos_df = pd.read_csv('./veiculos.csv')

## Exploração inicial dos dados

In [3]:
veiculos_df.head()

,Categoria,Cor,Pais de Origem,Ano Modelo,Ano Fabricação,Potencia,Quantidade de lugares,Unico dono?,Ja teve sinistro?,Ja foi carro de aplicativo?,Revisoes em dia?,Sistema avancado de Multimidia?,Tipo de Motorizacao,Kilometragem,Tipo de Transmissao,Tamanho do porta malas,Valor de Venda
0,4,azul,Alemanha,2026,2025,143,4,0,0,1,1,1,Híbrido,60459,3,430,112898.39
1,7,verde,Alemanha,2024,2024,541,5,0,0,1,1,0,Híbrido,105982,7,484,887822.26
2,2,prata,Japão,2026,2025,94,4,0,1,1,0,0,Flex,38626,3,321,55516.43
3,4,preto,Japão,2022,2021,159,4,1,1,0,0,1,Flex,91185,2,415,147030.87
4,2,azul,Coreia do Sul,2026,2025,114,4,0,0,0,0,1,Flex,26037,3,381,93719.67


In [4]:
veiculos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835 entries, 0 to 834
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Categoria                        835 non-null    int64  
 1   Cor                              835 non-null    object 
 2   Pais de Origem                   835 non-null    object 
 3   Ano Modelo                       835 non-null    int64  
 4   Ano Fabricação                   835 non-null    int64  
 5   Potencia                         835 non-null    int64  
 6   Quantidade de lugares            835 non-null    int64  
 7   Unico dono?                      835 non-null    int64  
 8   Ja teve sinistro?                835 non-null    int64  
 9   Ja foi carro de aplicativo?      835 non-null    int64  
 10  Revisoes em dia?                 835 non-null    int64  
 11  Sistema avancado de Multimidia?  835 non-null    int64  
 12  Tipo de Motorizacao   

In [5]:
veiculos_df.describe()

,Categoria,Ano Modelo,Ano Fabricação,Potencia,Quantidade de lugares,Unico dono?,Ja teve sinistro?,Ja foi carro de aplicativo?,Revisoes em dia?,Sistema avancado de Multimidia?,Kilometragem,Tipo de Transmissao,Tamanho do porta malas,Valor de Venda
count,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,8.350000e+02
mean,4.453892,2024.037126,2023.428743,307.476647,4.426347,0.492216,0.534132,0.462275,0.489820,0.489820,96439.767665,3.785629,395.049102,7.902733e+05
std,2.274606,1.402229,1.358211,320.806721,1.303114,0.500239,0.499133,0.498874,0.500196,0.500196,78575.237811,2.169836,131.769333,1.742400e+06
min,1.000000,2022.000000,2021.000000,70.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12611.000000,1.000000,80.000000,4.280000e+04
25%,2.000000,2023.000000,2022.000000,114.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,42289.500000,2.000000,308.000000,8.187166e+04
50%,4.000000,2024.000000,2024.000000,160.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,70509.000000,3.000000,428.000000,1.436343e+05
75%,6.000000,2025.000000,2025.000000,371.500000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,129393.500000,5.000000,490.500000,4.210006e+05
max,8.000000,2026.000000,2025.000000,1485.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,403947.000000,9.000000,600.000000,9.433305e+06


In [6]:
# Mostrar os valores únicos das variáveis categoricas

for col in veiculos_df.select_dtypes(include=['object']).columns:
    print(f'{col}: {veiculos_df[col].unique()}')

Cor: ['azul' 'verde' 'prata' 'preto' 'cinza' 'vermelho' 'branco']
Pais de Origem: ['Alemanha' 'Japão' 'Coreia do Sul' 'Inglaterra' 'França' 'Itália'
 'Estados Unidos' 'China']
Tipo de Motorizacao: ['Híbrido' 'Flex' 'Elétrico' 'Gasolina']


In [7]:
# Criar lista de variáveis categóricas
categorical_features = veiculos_df.select_dtypes(include=['object']).columns.tolist()
categorical_features

['Cor', 'Pais de Origem', 'Tipo de Motorizacao']

In [8]:
# Incluir variaveis categóricas adicionais
adicional_categorical_features = ['Categoria', 'Ano Modelo', 'Ano Fabricação', 
                                  'Unico dono?', 'Ja teve sinistro?',
                                  'Ja foi carro de aplicativo?','Revisoes em dia?',
                                  'Sistema avancado de Multimidia?','Tipo de Transmissao']
categorical_features.extend(adicional_categorical_features)
categorical_features

['Cor',
 'Pais de Origem',
 'Tipo de Motorizacao',
 'Categoria',
 'Ano Modelo',
 'Ano Fabricação',
 'Unico dono?',
 'Ja teve sinistro?',
 'Ja foi carro de aplicativo?',
 'Revisoes em dia?',
 'Sistema avancado de Multimidia?',
 'Tipo de Transmissao']

In [9]:
numerical_features = veiculos_df.select_dtypes(include=['number']).columns.tolist()
numerical_features

['Categoria',
 'Ano Modelo',
 'Ano Fabricação',
 'Potencia',
 'Quantidade de lugares',
 'Unico dono?',
 'Ja teve sinistro?',
 'Ja foi carro de aplicativo?',
 'Revisoes em dia?',
 'Sistema avancado de Multimidia?',
 'Kilometragem',
 'Tipo de Transmissao',
 'Tamanho do porta malas',
 'Valor de Venda']

In [10]:
# Remover target
numerical_features.remove('Valor de Venda')
# Remover features categoricas do tipo numérico
numerical_features = [feature for feature in numerical_features if feature not in categorical_features]
numerical_features

['Potencia', 'Quantidade de lugares', 'Kilometragem', 'Tamanho do porta malas']

In [11]:
# Variávei target
target = ['Valor de Venda']

## EDA

In [12]:
fig = px.histogram(veiculos_df,x='Valor de Venda', nbins=50,title="Distribuição do valor de venda")
fig.show()

In [13]:
# Distribuição das variáveis numéricas

for feature in numerical_features:
    fig = px.histogram(veiculos_df,nbins=50,x=feature, title=f'Distribuição de {feature}')
    fig.show()

In [14]:
# BoxPlot das variáveis numéricas

for feature in numerical_features:
    fig = px.box(veiculos_df,y=feature, title=f'Boxplot de {feature}')
    fig.show()

In [15]:
# Distribuição das variáveis categóricas
for feature in categorical_features:
    fig = px.histogram(veiculos_df, x=feature, title=f'Histograma da feature {feature}', color=feature)
    fig.show()

In [16]:
for feature in categorical_features:
    fig = px.box(veiculos_df,x=feature,y=target, title=f'Boxplot da feature {feature}', color=feature)
    fig.show()

## Preparar dados para correlações

In [17]:
# Transformar variáveis Categóricas originais do dataset, usando onehotencoding
df_veiculos_encoding = pd.get_dummies(veiculos_df, dtype=int)
df_veiculos_encoding.head()

,Categoria,Ano Modelo,Ano Fabricação,Potencia,Quantidade de lugares,Unico dono?,Ja teve sinistro?,Ja foi carro de aplicativo?,Revisoes em dia?,Sistema avancado de Multimidia?,...,Pais de Origem_Coreia do Sul,Pais de Origem_Estados Unidos,Pais de Origem_França,Pais de Origem_Inglaterra,Pais de Origem_Itália,Pais de Origem_Japão,Tipo de Motorizacao_Elétrico,Tipo de Motorizacao_Flex,Tipo de Motorizacao_Gasolina,Tipo de Motorizacao_Híbrido
0,4,2026,2025,143,4,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,7,2024,2024,541,5,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2026,2025,94,4,0,1,1,0,0,...,0,0,0,0,0,1,0,1,0,0
3,4,2022,2021,159,4,1,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0
4,2,2026,2025,114,4,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [18]:
fig = px.imshow(df_veiculos_encoding.corr(),text_auto=True, aspect='auto',title='Correlação entre as variáveis', width=1080,height=900)
fig.show()

In [19]:
# Transformar variáveis Categóricas da lista do dataset, usando onehotencoding
df_veiculos_encoding = pd.get_dummies(veiculos_df, columns=categorical_features, dtype=int)
df_veiculos_encoding.head()

,Potencia,Quantidade de lugares,Kilometragem,Tamanho do porta malas,Valor de Venda,Cor_azul,Cor_branco,Cor_cinza,Cor_prata,Cor_preto,...,Sistema avancado de Multimidia?_1,Tipo de Transmissao_1,Tipo de Transmissao_2,Tipo de Transmissao_3,Tipo de Transmissao_4,Tipo de Transmissao_5,Tipo de Transmissao_6,Tipo de Transmissao_7,Tipo de Transmissao_8,Tipo de Transmissao_9
0,143,4,60459,430,112898.39,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1,541,5,105982,484,887822.26,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,94,4,38626,321,55516.43,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,159,4,91185,415,147030.87,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
4,114,4,26037,381,93719.67,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [20]:
df_veiculos_encoding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835 entries, 0 to 834
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Potencia                           835 non-null    int64  
 1   Quantidade de lugares              835 non-null    int64  
 2   Kilometragem                       835 non-null    int64  
 3   Tamanho do porta malas             835 non-null    int64  
 4   Valor de Venda                     835 non-null    float64
 5   Cor_azul                           835 non-null    int64  
 6   Cor_branco                         835 non-null    int64  
 7   Cor_cinza                          835 non-null    int64  
 8   Cor_prata                          835 non-null    int64  
 9   Cor_preto                          835 non-null    int64  
 10  Cor_verde                          835 non-null    int64  
 11  Cor_vermelho                       835 non-null    int64  

In [21]:
fig = px.imshow(df_veiculos_encoding.corr(),text_auto=True, aspect='auto',title='Correlação entre as variáveis', width=1080,height=900)
fig.show()

## Preparar dados para treinamento da rede neural

In [22]:
df_veiculos_encoded = pd.get_dummies(veiculos_df, dtype=int)
df_veiculos_encoded

,Categoria,Ano Modelo,Ano Fabricação,Potencia,Quantidade de lugares,Unico dono?,Ja teve sinistro?,Ja foi carro de aplicativo?,Revisoes em dia?,Sistema avancado de Multimidia?,...,Pais de Origem_Coreia do Sul,Pais de Origem_Estados Unidos,Pais de Origem_França,Pais de Origem_Inglaterra,Pais de Origem_Itália,Pais de Origem_Japão,Tipo de Motorizacao_Elétrico,Tipo de Motorizacao_Flex,Tipo de Motorizacao_Gasolina,Tipo de Motorizacao_Híbrido
0,4,2026,2025,143,4,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,7,2024,2024,541,5,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2026,2025,94,4,0,1,1,0,0,...,0,0,0,0,0,1,0,1,0,0
3,4,2022,2021,159,4,1,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0
4,2,2026,2025,114,4,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,5,2026,2025,244,5,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
831,6,2022,2021,258,5,0,0,1,0,1,...,0,0,0,0,0,1,0,1,0,0
832,5,2023,2022,201,5,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,1
833,5,2024,2023,241,5,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [23]:
# Dividir o dataset entre X e y
X = np.array(df_veiculos_encoded.drop(target, axis=1))
y = np.array(df_veiculos_encoded[target])

In [24]:
X.shape,y.shape

((835, 32), (835, 1))

In [25]:
X,y

(array([[   4, 2026, 2025, ...,    0,    0,    1],
        [   7, 2024, 2024, ...,    0,    0,    1],
        [   2, 2026, 2025, ...,    1,    0,    0],
        ...,
        [   5, 2023, 2022, ...,    0,    0,    1],
        [   5, 2024, 2023, ...,    0,    0,    0],
        [   8, 2022, 2022, ...,    0,    0,    0]]),
 array([[ 112898.39],
        [ 887822.26],
        [  55516.43],
        [ 147030.87],
        [  93719.67],
        [ 360835.36],
        [ 595305.35],
        [  62400.74],
        [1475001.6 ],
        [  42800.  ],
        [7003596.07],
        [ 104147.1 ],
        [ 104476.44],
        [ 135115.73],
        [ 296318.67],
        [  53277.76],
        [  42800.  ],
        [  81175.83],
        [1812181.15],
        [ 190199.07],
        [  54839.79],
        [ 934840.25],
        [ 343121.77],
        [ 246775.47],
        [1380840.49],
        [8509786.77],
        [6961984.36],
        [6907950.63],
        [  70242.  ],
        [9433305.05],
        [  46919.05

In [26]:
# Dividir entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=42,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test, test_size=0.5,random_state=42,shuffle=True)

In [27]:
X_train.shape, X_test.shape, X_val.shape

((417, 32), (209, 32), (209, 32))

In [ ]:
class VeiculosDataset(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    

In [29]:
# Criar datasets
dataset_train = VeiculosDataset(X_train,y_train)
dataset_val = VeiculosDataset(X_val,y_val)
dataset_test = VeiculosDataset(X_test, y_test)

In [30]:
# Criar dataLoaders
dataloader_train = DataLoader(dataset_train, batch_size=32)
dataloader_val = DataLoader(dataset_val, batch_size=32)
dataloader_test = DataLoader(dataset_test, batch_size=32)

## Definir a Arquitetura da Rede

In [31]:
# Criar uma arquitetura de rede neural com Pytorch
class NeuralNetwork(nn.Module):
    def __init__(self, input_size,hidden_layer_sizes=[128,64,32,16],output_size=1):
        super(NeuralNetwork,self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_layer_sizes[0])
        self.layer2 = nn.Linear(hidden_layer_sizes[0], hidden_layer_sizes[1])
        self.layer3 = nn.Linear(hidden_layer_sizes[1], hidden_layer_sizes[2])
        self.layer4 = nn.Linear(hidden_layer_sizes[2], hidden_layer_sizes[3])
        self.output_layer = nn.Linear(hidden_layer_sizes[3], output_size)
        self.relu = nn.ReLU()

    def forward(self,X):
        X = self.relu(self.layer1(X))
        X = self.relu(self.layer2(X))
        X = self.relu(self.layer3(X))
        X = self.relu(self.layer4(X))
        X = self.output_layer(X)
        return X

In [ ]:
# Instanciar o modleo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(input_size=X_train.shape[1], hidden_layer_sizes=[64,32,16,8], output_size=1).to(device)


In [38]:
# Visualizar a arquitetura do modelo
summary(model, input_size=(X_train.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]           2,112
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 32]           2,080
              ReLU-4                   [-1, 32]               0
            Linear-5                   [-1, 16]             528
              ReLU-6                   [-1, 16]               0
            Linear-7                    [-1, 8]             136
              ReLU-8                    [-1, 8]               0
            Linear-9                    [-1, 1]               9
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.02
-----------------------------------------------

## Treinar a rede neural

In [45]:
# Treinar a rede
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
NUM_EPOCHS = 200
train_losses = []
val_losses = []
model.to(device)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_train_loss = 0.0

    for data in dataloader_train:
        # Zerar os gradientes
        optimizer.zero_grad()

        # Dividir entre input e output
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward Pass
        outputs = model.forward(inputs) # Predição
        loss = criterion(outputs, targets) # Caculo do loss

        # BackPropagation
        loss.backward() # Calcula gradientes
        optimizer.step() # Atualiza pesos

        running_train_loss += loss.item()
    epoch_train_loss = running_train_loss / len(dataloader_train)
    train_losses.append(epoch_train_loss)

    # Fase de Validação
    model.eval()
    running_val_losses = 0.0

    with torch.no_grad():
        for data in dataloader_val:
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model.forward(inputs)
            loss = criterion(outputs, targets)
            
            running_val_losses += loss.item()
        epoch_val_losses = running_val_losses/ len(dataloader_val)
        val_losses.append(epoch_val_losses)
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}\nPerda no teste: {running_train_loss:.6f}\nPerda na validação {running_val_losses:.6f}")



Epoch: 0
Perda no teste: nan
Perda na validação nan
Epoch: 10
Perda no teste: nan
Perda na validação nan
Epoch: 20
Perda no teste: nan
Perda na validação nan
Epoch: 30
Perda no teste: nan
Perda na validação nan
Epoch: 40
Perda no teste: nan
Perda na validação nan
Epoch: 50
Perda no teste: nan
Perda na validação nan
Epoch: 60
Perda no teste: nan
Perda na validação nan
Epoch: 70
Perda no teste: nan
Perda na validação nan
Epoch: 80
Perda no teste: nan
Perda na validação nan
Epoch: 90
Perda no teste: nan
Perda na validação nan
Epoch: 100
Perda no teste: nan
Perda na validação nan
Epoch: 110
Perda no teste: nan
Perda na validação nan
Epoch: 120
Perda no teste: nan
Perda na validação nan
Epoch: 130
Perda no teste: nan
Perda na validação nan
Epoch: 140
Perda no teste: nan
Perda na validação nan
Epoch: 150
Perda no teste: nan
Perda na validação nan
Epoch: 160
Perda no teste: nan
Perda na validação nan
Epoch: 170
Perda no teste: nan
Perda na validação nan
Epoch: 180
Perda no teste: nan
Perda na